# 1.1.1 财务报表调整框架（示例：比亚迪 BYD，002594.SZ）

本 notebook 用 **Python** 演示如何把“气候转型风险/碳市场冲击”映射到财务报表，并完成：

1. 基准情景（Baseline）财务预测
2. 利润表科目调整（收入、成本、税金及附加等）
3. 资产负债表科目调整（现金、存货、固定资产、无形资产等）
4. 关键指标测算（ROA/ROE/流动性/CCC/Altman Z 值）
5. 情景结果分析与可视化

> 说明：这里的调整是“压力测试/情景分析”的教学版实现，用于展示方法与勾稽关系，
> 并不等同于审计口径的会计处理。


## (1) 财务调整的国际准则依据（用于识别影响维度）

- **IAS 1**：重要性判断与披露（减值、诉讼、修复义务、需求变化等）
- **IAS 2**：存货计量（成本与可变现净值变化；存货跌价）
- **IAS 16**：固定资产（使用寿命/残值/折旧方法；减值测试）
- **IAS 38**：无形资产（技术迭代导致减值；碳排放权交易在我国常按无形资产处理）
- **IAS 36**：资产减值（未来现金流/折现率/可回收金额变化触发减值）
- **IFRS 13**：公允价值计量（市场参与者假设变化影响公允价值）

下面我们把这些“可能的会计影响”抽象成可操作的模型参数（例如：收入增长冲击、碳成本率、
存货/固定资产/无形资产减值比例、资本开支上升等），并把情景冲击逐步传导到三张表。


In [ ]:
from __future__ import annotations

import os
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

def _fmt(x: float) -> str:
    ax = abs(x)
    if ax < 1:
        return f"{x:,.4f}"
    if ax < 100:
        return f"{x:,.2f}"
    return f"{x:,.0f}"

pd.options.display.float_format = _fmt

# Network-dependent cells are optional.
RUN_ONLINE = False  # set True if you want to fetch live data
RUN_ONLINE = RUN_ONLINE or (os.environ.get('RUN_ONLINE', '0') == '1')

# This notebook lives under `materials/`. Depending on how Jupyter is launched,
# the kernel's working directory may be repo root or `materials/`.
DATA_DIR = Path('data') if Path('data').exists() else Path('materials/data')
CHINA_AUTO_DIR = DATA_DIR / 'china_auto'

SYMBOL = 'sz002594'  # BYD: 002594.SZ

(CHINA_AUTO_DIR.exists(), (CHINA_AUTO_DIR / f"{SYMBOL}_profit_sheet.csv.gz").exists())


In [ ]:
# Optional: refresh statements via akshare (network)
# Keep RUN_ONLINE=False by default for reproducibility in class.
if RUN_ONLINE:
    import akshare as ak

    CHINA_AUTO_DIR.mkdir(parents=True, exist_ok=True)

    ak.stock_profit_sheet_by_report_em(symbol=SYMBOL).to_csv(
        CHINA_AUTO_DIR / f"{SYMBOL}_profit_sheet.csv.gz", index=False, compression='gzip'
    )
    ak.stock_balance_sheet_by_report_em(symbol=SYMBOL).to_csv(
        CHINA_AUTO_DIR / f"{SYMBOL}_balance_sheet.csv.gz", index=False, compression='gzip'
    )
    ak.stock_cash_flow_sheet_by_report_em(symbol=SYMBOL).to_csv(
        CHINA_AUTO_DIR / f"{SYMBOL}_cash_flow_sheet.csv.gz", index=False, compression='gzip'
    )

    print('Refreshed:', SYMBOL)
else:
    print('RUN_ONLINE=False (using local cached files)')


## (2) 调整方法与实施步骤：先准备基准财务数据（BYD 年报口径）


In [ ]:
def load_em_statements(symbol: str, data_dir: Path):
    profit = pd.read_csv(data_dir / f"{symbol}_profit_sheet.csv.gz", compression='gzip')
    balance = pd.read_csv(data_dir / f"{symbol}_balance_sheet.csv.gz", compression='gzip')
    cash = pd.read_csv(data_dir / f"{symbol}_cash_flow_sheet.csv.gz", compression='gzip')

    for df in (profit, balance, cash):
        df['REPORT_DATE'] = pd.to_datetime(df['REPORT_DATE'], errors='coerce')
        df.dropna(subset=['REPORT_DATE'], inplace=True)
        df.sort_values('REPORT_DATE', inplace=True)
        df.drop_duplicates('REPORT_DATE', keep='last', inplace=True)

    return profit, balance, cash


def annual(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    # Annual reports correspond to year-end (December 31)
    out = out[out['REPORT_DATE'].dt.strftime('%m-%d').eq('12-31')]
    return out.set_index('REPORT_DATE').sort_index()


profit_raw, balance_raw, cash_raw = load_em_statements(SYMBOL, CHINA_AUTO_DIR)

profit_a = annual(profit_raw)
balance_a = annual(balance_raw)
cash_a = annual(cash_raw)

(len(profit_a), len(balance_a), len(cash_a), profit_a.index.max(), balance_a.index.max(), cash_a.index.max())


In [ ]:
def pick(df: pd.DataFrame, col: str) -> pd.Series:
    if col not in df.columns:
        return pd.Series(index=df.index, dtype='float64')
    return pd.to_numeric(df[col], errors='coerce')


net_col = 'PARENT_NETPROFIT' if 'PARENT_NETPROFIT' in profit_a.columns else 'NETPROFIT'

is_hist = pd.DataFrame(
    {
        'Revenue': pick(profit_a, 'TOTAL_OPERATE_INCOME'),
        'COGS': pick(profit_a, 'OPERATE_COST'),
        'Tax & Surcharges': pick(profit_a, 'OPERATE_TAX_ADD'),
        'Selling Expense': pick(profit_a, 'SALE_EXPENSE'),
        'Admin Expense': pick(profit_a, 'MANAGE_EXPENSE'),
        'R&D Expense': pick(profit_a, 'RESEARCH_EXPENSE'),
        'Finance Expense': pick(profit_a, 'FINANCE_EXPENSE'),
        'Operating Profit (reported)': pick(profit_a, 'OPERATE_PROFIT'),
        'Pretax Profit (reported)': pick(profit_a, 'TOTAL_PROFIT'),
        'Income Tax': pick(profit_a, 'INCOME_TAX'),
        'Net Profit': pick(profit_a, net_col),
    }
).sort_index()

bs_hist = pd.DataFrame(
    {
        'Cash': pick(balance_a, 'MONETARYFUNDS'),
        'Accounts Receivable': pick(balance_a, 'ACCOUNTS_RECE'),
        'Inventory': pick(balance_a, 'INVENTORY'),
        'Accounts Payable': pick(balance_a, 'ACCOUNTS_PAYABLE'),
        'PPE': pick(balance_a, 'FIXED_ASSET') + pick(balance_a, 'CIP'),
        'Intangibles': pick(balance_a, 'INTANGIBLE_ASSET'),
        'Total Current Assets': pick(balance_a, 'TOTAL_CURRENT_ASSETS'),
        'Total Current Liabilities': pick(balance_a, 'TOTAL_CURRENT_LIAB'),
        'Total Assets': pick(balance_a, 'TOTAL_ASSETS'),
        'Total Liabilities': pick(balance_a, 'TOTAL_LIABILITIES'),
        'Total Equity': pick(balance_a, 'TOTAL_EQUITY'),
        'Share Capital': pick(balance_a, 'SHARE_CAPITAL'),
        'Retained Earnings': pick(balance_a, 'UNASSIGN_RPOFIT'),
    }
).sort_index()

cf_hist = pd.DataFrame(
    {
        'Operating Cash Flow': pick(cash_a, 'NETCASH_OPERATE'),
        'Capex (cash paid)': pick(cash_a, 'CONSTRUCT_LONG_ASSET'),
        'Depreciation': pick(cash_a, 'FA_IR_DEPR'),
        'Amortization (IA)': pick(cash_a, 'IA_AMORTIZE'),
    }
).sort_index()

display(is_hist.tail(6))
display(bs_hist.tail(6))
display(cf_hist.tail(6))


## 第一步：构建基准情景下的财务预测指标

这里用历史 3 年均值/中位数去估计一些关键比率（教学版简化）：

- 利润表：成本率、费用率、税率
- 资产负债表：应收/存货/应付周转天数（DSO/DIO/DPO）、杠杆率
- 现金流：资本开支率、折旧与摊销率

你可以把这些参数替换成更严谨的宏观/行业预测与情景模型输出。


In [ ]:
def _safe_mean(x: pd.Series, default: float) -> float:
    x = pd.to_numeric(x, errors='coerce').replace([np.inf, -np.inf], np.nan).dropna()
    return float(x.mean()) if len(x) else float(default)


common = is_hist.index.intersection(bs_hist.index).intersection(cf_hist.index)
base_date = common.max()
hist_years = common[-3:] if len(common) >= 3 else common

rev = is_hist.loc[hist_years, 'Revenue']
cogs = is_hist.loc[hist_years, 'COGS']

# Baseline growth: median YoY from the last few annual points
rev_growth = is_hist['Revenue'].pct_change(fill_method=None).replace([np.inf, -np.inf], np.nan).dropna().tail(5).median()
rev_growth = float(rev_growth) if pd.notna(rev_growth) else 0.10

ratios = {
    'cogs_ratio': _safe_mean(cogs / rev, 0.80),
    'tax_surch_ratio': _safe_mean(is_hist.loc[hist_years, 'Tax & Surcharges'] / rev, 0.01),
    'selling_ratio': _safe_mean(is_hist.loc[hist_years, 'Selling Expense'] / rev, 0.03),
    'admin_ratio': _safe_mean(is_hist.loc[hist_years, 'Admin Expense'] / rev, 0.02),
    'rnd_ratio': _safe_mean(is_hist.loc[hist_years, 'R&D Expense'] / rev, 0.03),
    'finance_ratio': _safe_mean(is_hist.loc[hist_years, 'Finance Expense'] / rev, 0.01),
}

tax_rate = (is_hist.loc[hist_years, 'Income Tax'] / is_hist.loc[hist_years, 'Pretax Profit (reported)']).replace(
    [np.inf, -np.inf], np.nan
)
tax_rate = float(tax_rate.clip(lower=0, upper=0.35).dropna().mean()) if tax_rate.dropna().any() else 0.25

leverage = _safe_mean(bs_hist.loc[hist_years, 'Total Liabilities'] / bs_hist.loc[hist_years, 'Total Assets'], 0.60)

dso = _safe_mean(bs_hist.loc[hist_years, 'Accounts Receivable'] / rev * 365, 60)
dio = _safe_mean(bs_hist.loc[hist_years, 'Inventory'] / cogs * 365, 80)
dpo = _safe_mean(bs_hist.loc[hist_years, 'Accounts Payable'] / cogs * 365, 90)

capex_ratio = _safe_mean(cf_hist.loc[hist_years, 'Capex (cash paid)'] / rev, 0.08)
depr_rate = _safe_mean(cf_hist.loc[hist_years, 'Depreciation'] / bs_hist.loc[hist_years, 'PPE'], 0.08)
amort_rate = _safe_mean(cf_hist.loc[hist_years, 'Amortization (IA)'] / bs_hist.loc[hist_years, 'Intangibles'], 0.10)

# Other balance sheet buckets: keep as (roughly) stable ratios to Revenue
other_ca = bs_hist.loc[hist_years, 'Total Current Assets'] - bs_hist.loc[hist_years, ['Cash', 'Accounts Receivable', 'Inventory']].sum(axis=1)
other_cl = bs_hist.loc[hist_years, 'Total Current Liabilities'] - bs_hist.loc[hist_years, 'Accounts Payable']
other_nca = (
    bs_hist.loc[hist_years, 'Total Assets']
    - bs_hist.loc[hist_years, 'Total Current Assets']
    - bs_hist.loc[hist_years, 'PPE']
    - bs_hist.loc[hist_years, 'Intangibles']
)

other_ca_ratio = _safe_mean(other_ca / rev, 0.05)
other_cl_ratio = _safe_mean(other_cl / rev, 0.08)
other_nca_ratio = _safe_mean(other_nca / rev, 0.10)

cash_min_ratio = _safe_mean(bs_hist.loc[hist_years, 'Cash'] / rev, 0.10)

payout_ratio = 0.20  # 简化：分红率（可自行替换）

horizon = 5
future = [pd.Timestamp(y, 12, 31) for y in range(base_date.year + 1, base_date.year + 1 + horizon)]

drivers_base = pd.DataFrame(index=future)
drivers_base['rev_growth'] = rev_growth
drivers_base['tax_rate'] = tax_rate
drivers_base['leverage'] = leverage
drivers_base['payout_ratio'] = payout_ratio

for k, v in ratios.items():
    drivers_base[k] = v

drivers_base['carbon_cost_ratio'] = 0.0
drivers_base['capex_ratio'] = capex_ratio
drivers_base['depr_rate'] = depr_rate
drivers_base['amort_rate'] = amort_rate

drivers_base['dso'] = dso
drivers_base['dio'] = dio
drivers_base['dpo'] = dpo

drivers_base['other_ca_ratio'] = other_ca_ratio
drivers_base['other_cl_ratio'] = other_cl_ratio
drivers_base['other_nca_ratio'] = other_nca_ratio

drivers_base['cash_min_ratio'] = cash_min_ratio

# One-off impairments / write-downs (set by scenarios)
drivers_base['inv_write_down_pct'] = 0.0
drivers_base['ppe_impairment_pct'] = 0.0
drivers_base['intang_impairment_pct'] = 0.0

display(
    pd.DataFrame(
        {
            'baseline': pd.Series(
                {**ratios, 'tax_rate': tax_rate, 'leverage': leverage, 'rev_growth': rev_growth, 'cash_min_ratio': cash_min_ratio}
            )
        }
    )
)
display(drivers_base)


## 第二步：实施利润表科目的调整（将情景变量映射到科目）

示例映射（可按你的情景模型输出替换）：

- **需求/价格压力** → `rev_growth` 下调（收入增速冲击）
- **碳价/碳配额成本** → `carbon_cost_ratio` 上调（以收入的比例近似）
- **政策与合规成本** → 费用率上调（这里合并到费用率；也可拆到税金及附加）
- **极端天气/供应链扰动** → 周转天数变差（DSO/DIO 上升）
- **搁浅资产/技术迭代** → `ppe_impairment_pct` / `intang_impairment_pct` 上升（减值损失）
- **存货可变现净值下降** → `inv_write_down_pct`（存货跌价）


In [ ]:
def make_scenario_drivers(base: pd.DataFrame, scenario: str) -> pd.DataFrame:
    d = base.copy()

    if scenario == 'BASE':
        return d

    if scenario == 'S1_CARBON_COST':
        # Carbon cost gradually rises from 0.2% to 1.0% of revenue
        d['carbon_cost_ratio'] = np.linspace(0.002, 0.010, len(d))
        # More transition capex
        d['capex_ratio'] = d['capex_ratio'] + 0.01
        return d

    if scenario == 'S2_DEMAND_SHOCK':
        # Demand shock: revenue growth -6pp; working capital turns worse
        d['rev_growth'] = (d['rev_growth'] - 0.06).clip(lower=-0.20)
        d['dso'] = d['dso'] + 15
        d['dio'] = d['dio'] + 10
        d['inv_write_down_pct'] = 0.01
        return d

    if scenario == 'S3_COMBINED':
        d = make_scenario_drivers(d, 'S1_CARBON_COST')
        d = make_scenario_drivers(d, 'S2_DEMAND_SHOCK')
        # One-off impairments in the first forecast year
        first = d.index.min()
        d.loc[first, 'ppe_impairment_pct'] = 0.05
        d.loc[first, 'intang_impairment_pct'] = 0.10
        d.loc[first, 'inv_write_down_pct'] = 0.03
        # Extra transition capex on top
        d['capex_ratio'] = d['capex_ratio'] + 0.01
        return d

    raise ValueError(f'Unknown scenario: {scenario}')


scenarios = ['BASE', 'S1_CARBON_COST', 'S2_DEMAND_SHOCK', 'S3_COMBINED']
drivers_by_scn = {s: make_scenario_drivers(drivers_base, s) for s in scenarios}

for s in scenarios:
    print(s)
    display(drivers_by_scn[s].head(2))


## 第三步：实施资产负债表科目的调整（用会计恒等式与勾稽关系传导）

本节用一个简化的三表联动：

- 利润表的变化会影响 **净利润** → 进而更新 **留存收益/所有者权益**
- 存货跌价、固定资产/无形资产减值会同时影响 **利润表（减值损失）** 与 **资产负债表（资产减少）**
- 用“**目标杠杆率**（历史均值）”来闭合模型：
  - `Assets_target = Equity / (1 - leverage_target)`
  - `Cash_min = Revenue * cash_min_ratio`
  - `Total Assets = max(Assets_target, Noncash Assets + Cash_min)`
  - `Total Liabilities = Total Assets - Equity`

你也可以把“目标杠杆率”换成更细的融资假设（新增借款、发行股票、分红政策等）。


In [ ]:
def run_projection(
    base_date: pd.Timestamp,
    is_hist: pd.DataFrame,
    bs_hist: pd.DataFrame,
    drivers: pd.DataFrame,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    base_bs = bs_hist.loc[base_date]
    share_capital = float(base_bs['Share Capital'])
    retained = float(base_bs['Retained Earnings'])
    other_equity = float(base_bs['Total Equity'] - base_bs['Share Capital'] - base_bs['Retained Earnings'])

    revenue_prev = float(is_hist.loc[base_date, 'Revenue'])
    ppe_prev = float(base_bs['PPE'])
    intang_prev = float(base_bs['Intangibles'])

    is_rows = []
    bs_rows = []

    for dt, p in drivers.iterrows():
        revenue = revenue_prev * (1 + float(p['rev_growth']))

        cogs_base = revenue * float(p['cogs_ratio'])
        carbon_cost = revenue * float(p.get('carbon_cost_ratio', 0.0))
        cogs_total = cogs_base + carbon_cost

        tax_surch = revenue * float(p['tax_surch_ratio'])
        selling = revenue * float(p['selling_ratio'])
        admin = revenue * float(p['admin_ratio'])
        rnd = revenue * float(p['rnd_ratio'])
        finance = revenue * float(p['finance_ratio'])

        # Working capital (end-of-year approximation)
        ar = revenue / 365.0 * float(p['dso'])
        ap = cogs_total / 365.0 * float(p['dpo'])

        inv_pre = cogs_total / 365.0 * float(p['dio'])
        inv_write_down = inv_pre * float(p.get('inv_write_down_pct', 0.0))
        inventory = inv_pre - inv_write_down

        other_ca = revenue * float(p['other_ca_ratio'])
        other_cl = revenue * float(p['other_cl_ratio'])
        other_nca = revenue * float(p['other_nca_ratio'])

        # PPE and intangibles roll-forward (very simplified)
        capex = revenue * float(p['capex_ratio'])
        depreciation = ppe_prev * float(p['depr_rate'])
        ppe_impairment = ppe_prev * float(p.get('ppe_impairment_pct', 0.0))
        ppe = max(0.0, ppe_prev + capex - depreciation - ppe_impairment)

        amortization = intang_prev * float(p['amort_rate'])
        intang_impairment = intang_prev * float(p.get('intang_impairment_pct', 0.0))
        intang = max(0.0, intang_prev - amortization - intang_impairment)

        impairment_loss = inv_write_down + ppe_impairment + intang_impairment

        ebit = (
            revenue
            - cogs_total
            - tax_surch
            - selling
            - admin
            - rnd
            - finance
            - impairment_loss
        )
        pretax = ebit
        tax = max(0.0, pretax * float(p['tax_rate']))
        net = pretax - tax

        dividends = float(p['payout_ratio']) * max(net, 0.0)
        retained = retained + net - dividends
        total_equity = share_capital + other_equity + retained

        leverage_target = float(p['leverage'])
        assets_target = total_equity / (1.0 - leverage_target)

        noncash_assets = ar + inventory + other_ca + ppe + intang + other_nca
        cash_min = revenue * float(p.get('cash_min_ratio', 0.0))
        total_assets = max(assets_target, noncash_assets + cash_min)
        cash = total_assets - noncash_assets

        total_liab = total_assets - total_equity
        leverage_implied = total_liab / total_assets if total_assets else np.nan

        # Keep AP as operating liability; adjust 'Other Current Liabilities' if needed
        other_cl_adj = other_cl
        if ap + other_cl_adj > total_liab:
            other_cl_adj = max(0.0, total_liab - ap)
        current_liab = ap + other_cl_adj
        noncurrent_liab = total_liab - current_liab

        is_rows.append(
            {
                'Year': dt,
                'Revenue': revenue,
                'COGS (base)': cogs_base,
                'Carbon Cost': carbon_cost,
                'COGS (total)': cogs_total,
                'Tax & Surcharges': tax_surch,
                'Selling Expense': selling,
                'Admin Expense': admin,
                'R&D Expense': rnd,
                'Finance Expense': finance,
                'Impairment Loss': impairment_loss,
                'EBIT (model)': ebit,
                'Income Tax': tax,
                'Net Profit': net,
                'Dividends (assumed)': dividends,
            }
        )

        bs_rows.append(
            {
                'Year': dt,
                'Cash': cash,
                'Accounts Receivable': ar,
                'Inventory': inventory,
                'Other Current Assets': other_ca,
                'PPE': ppe,
                'Intangibles': intang,
                'Other Noncurrent Assets': other_nca,
                'Total Assets': total_assets,
                'Accounts Payable': ap,
                'Other Current Liabilities': other_cl_adj,
                'Total Current Liabilities': current_liab,
                'Noncurrent Liabilities (plug)': noncurrent_liab,
                'Total Liabilities': total_liab,
                'Leverage (implied)': leverage_implied,
                'Share Capital': share_capital,
                'Retained Earnings': retained,
                'Other Equity': other_equity,
                'Total Equity': total_equity,
            }
        )

        revenue_prev = revenue
        ppe_prev = ppe
        intang_prev = intang

    is_fcst = pd.DataFrame(is_rows).set_index('Year')
    bs_fcst = pd.DataFrame(bs_rows).set_index('Year')

    # Basic sanity checks
    check = (bs_fcst['Total Assets'] - bs_fcst['Total Liabilities'] - bs_fcst['Total Equity']).abs().max()
    if check > 1e-2:
        print('Warning: balance sheet not balanced. max abs gap =', check)

    return is_fcst, bs_fcst


base_date


In [ ]:
results = {}
for s in scenarios:
    is_fcst, bs_fcst = run_projection(base_date, is_hist, bs_hist, drivers_by_scn[s])
    results[s] = {'is': is_fcst, 'bs': bs_fcst}

display(results['BASE']['is'].head())
display(results['BASE']['bs'].head())


## 第四步：关键财务指标测算（含 Z 值）

示例 KPI：

- 盈利能力：ROA、ROE、Operating Margin
- 流动性：Current Ratio、现金余额
- 营运效率：DSO/DIO/DPO、现金周转天数（CCC）
- 信用风险：Altman Z'（Z-prime，常用于私有制造业版本，使用账面权益而非市值）

> Altman Z 指标有多种版本（Z、Z'、Z''），不同国家/行业适用性不同。这里用于课堂演示。 


In [ ]:
def compute_kpis(is_df: pd.DataFrame, bs_df: pd.DataFrame) -> pd.DataFrame:
    out = pd.DataFrame(index=is_df.index)

    ta = bs_df['Total Assets']
    te = bs_df['Total Equity']
    tl = bs_df['Total Liabilities']

    out['ROA'] = is_df['Net Profit'] / ta
    out['ROE'] = is_df['Net Profit'] / te
    out['Operating Margin'] = is_df['EBIT (model)'] / is_df['Revenue']

    ca = (
        bs_df['Cash']
        + bs_df['Accounts Receivable']
        + bs_df['Inventory']
        + bs_df['Other Current Assets']
    )
    cl = bs_df['Total Current Liabilities']
    out['Current Ratio'] = ca / cl

    out['Leverage'] = tl / ta

    out['DSO'] = bs_df['Accounts Receivable'] / is_df['Revenue'] * 365
    out['DIO'] = bs_df['Inventory'] / is_df['COGS (total)'] * 365
    out['DPO'] = bs_df['Accounts Payable'] / is_df['COGS (total)'] * 365
    out['CCC'] = out['DSO'] + out['DIO'] - out['DPO']

    # Altman Z' (Z-prime) for (private) manufacturing firms:
    # Z' = 0.717*X1 + 0.847*X2 + 3.107*X3 + 0.420*X4 + 0.998*X5
    wc = ca - cl
    x1 = wc / ta
    x2 = bs_df['Retained Earnings'] / ta
    x3 = is_df['EBIT (model)'] / ta
    x4 = te / tl
    x5 = is_df['Revenue'] / ta
    out["Altman Z (Z')"] = 0.717 * x1 + 0.847 * x2 + 3.107 * x3 + 0.420 * x4 + 0.998 * x5

    out['Cash'] = bs_df['Cash']
    return out.replace([np.inf, -np.inf], np.nan)


kpis = {s: compute_kpis(results[s]['is'], results[s]['bs']) for s in scenarios}
kpis_table = pd.concat(kpis, axis=1)
display(kpis_table.tail())


In [ ]:
def plot_kpi(metric: str, ylabel: str | None = None):
    plt.figure(figsize=(10, 4))
    for s in scenarios:
        y = kpis[s][metric]
        plt.plot(y.index.year, y.values, marker='o', linewidth=2, label=s)
    plt.title(metric)
    plt.ylabel(ylabel or metric)
    plt.grid(True, alpha=0.3)
    plt.legend(loc='best')
    plt.show()


plot_kpi('ROE', 'ROE')
plot_kpi('Current Ratio', 'Current Ratio')
plot_kpi("Altman Z (Z')", "Altman Z (Z')")


## 第五步：结果分析（如何读图与落地到研究/报告）

建议在报告中把“情景 → 科目 → 指标”的链路写清楚：

1. 情景假设：碳价路径、需求冲击、技术替代速度、政策约束等。
2. 映射规则：哪些假设影响收入？哪些影响成本/费用？哪些触发减值/搁浅资产？
3. 三表传导：利润变化如何进入权益？资产减值如何同时影响利润与资产？现金是否成为“压力点”？
4. 指标解读：盈利能力、流动性、周转效率、信用风险（Z 值）的变化与原因。
5. 敏感性：对关键参数（碳成本率、减值比例、周转天数、杠杆率）的敏感性分析。

下一步可扩展：
- 更精细的科目拆分（营业成本分解为能源/原材料/人工；费用与资本化规则等）
- 用情景模型输出替代“历史均值比率”
- 将融资假设显式化（新增借款、再融资成本、权益融资）
- 若需要 IFRS/IAS 口径严谨处理：将减值测试与公允价值计量假设单独建模，并补充披露文本草稿
